# Запуск NGROK туннеля с 1м коллабом, соединение с телеграммом и общение с ним

In [ ]:
!pip install python-telegram-bot matplotlib
!pip install python-telegram-bot --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.7/662.7 kB 10.5 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from telegram import Update, InputFile
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [ ]:
def convert_strings_to_datetime(df):

    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    return df

def plot_combined_stock_prices(stock_data, prediction_df):

    try:

        if "date" in stock_data.columns and "value" in stock_data.columns \
           and "date" in prediction_df.columns and "predicted_value" in prediction_df.columns:


            stock_data["date"] = pd.to_datetime(stock_data["date"])
            prediction_df["date"] = pd.to_datetime(prediction_df["date"])


            recent_stock_data = stock_data.tail(60)


            plt.figure(figsize=(18, 7))
            plt.plot(recent_stock_data["date"], recent_stock_data["value"], label="Исторические данные", color="blue")
            plt.plot(prediction_df["date"], prediction_df["predicted_value"], label="Предсказания", color="green", linestyle="--")


            plt.title("График курса акций", fontsize=16)
            plt.xlabel("Дата", fontsize=12)
            plt.ylabel("Цена акций (в рублях)", fontsize=12)
            plt.grid(True, linestyle="--", alpha=0.5)
            plt.legend(fontsize=12)
            plt.tight_layout()
            plt.show()
        else:
            print("Ошибка: В одном из датафреймов отсутствуют необходимые колонки 'date' и 'value/predicted_value'.")
    except Exception as e:
        print(f"Ошибка при построении графика: {e}")

def plot_stock_prices(dataframe):

    try:

        if "date" in dataframe.columns and "value" in dataframe.columns:

            dataframe["date"] = pd.to_datetime(dataframe["date"])


            plt.figure(figsize=(14, 7))
            plt.plot(dataframe["date"], dataframe["value"], label="Цена акций", color="blue")
            plt.title("График цены акций", fontsize=16)
            plt.xlabel("Дата", fontsize=12)
            plt.ylabel("Цена акций (в рублях)", fontsize=12)
            plt.grid(True, linestyle="--", alpha=0.5)
            plt.legend(fontsize=12)
            plt.show()
        else:
            print("Ошибка: В датафрейме отсутствуют необходимые колонки 'date' и 'value'.")
    except Exception as e:
        print(f"Ошибка при построении графика: {e}")

In [ ]:
TELEGRAM_TOKEN = "7390352078:AAGa1MYBic_-tXNUrqRLEKmSZPm5X1ojZEM"


SERVER_URL = "https://3cc7-34-56-131-214.ngrok-free.app"

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

def plot_combined_stock_prices(stock_data, prediction_df):
    try:
        if "date" in stock_data.columns and "value" in stock_data.columns \
           and "date" in prediction_df.columns and "predicted_value" in prediction_df.columns:
            stock_data["date"] = pd.to_datetime(stock_data["date"])
            prediction_df["date"] = pd.to_datetime(prediction_df["date"])

            recent_stock_data = stock_data.tail(60)

            plt.figure(figsize=(18, 7))
            plt.plot(recent_stock_data["date"], recent_stock_data["value"], label="Исторические данные", color="blue")
            plt.plot(prediction_df["date"], prediction_df["predicted_value"], label="Предсказания", color="green", linestyle="--")

            plt.title("График курса акций", fontsize=16)
            plt.xlabel("Дата", fontsize=12)
            plt.ylabel("Цена акций (в рублях)", fontsize=12)
            plt.grid(True, linestyle="--", alpha=0.5)
            plt.legend(fontsize=12)
            plt.tight_layout()

            plot_path = "forecast_plot.png"
            plt.savefig(plot_path)
            plt.close()
            return plot_path
        else:
            raise ValueError("Отсутствуют необходимые колонки 'date' и 'value/predicted_value'.")
    except Exception as e:
        raise RuntimeError(f"Ошибка построения графика: {e}")


async def handle_ticker(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        ticker = update.message.text.strip().upper()
        await update.message.reply_text(f"Получен тикер: {ticker}. Выполняется прогноз...")

        response = requests.get(f"{SERVER_URL}/{ticker}")
        if response.status_code != 200:
            await update.message.reply_text(f"Ошибка сервера: {response.status_code}")
            return

        data = response.json()
        stock_data = pd.DataFrame(data['df1'])
        prediction_df = pd.DataFrame(data['df2'])
        prediction_df.rename(columns={"value": "predicted_value"}, inplace=True)

        plot_path = plot_combined_stock_prices(stock_data, prediction_df)

        with open(plot_path, "rb") as plot_file:
            await update.message.reply_photo(photo=InputFile(plot_file), caption=f"Прогноз цены для {ticker}")

    except Exception as e:
        logger.error(f"Ошибка: {e}")
        await update.message.reply_text(f"Произошла ошибка: {str(e)}")

# Обработчик команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Добро пожаловать! Отправьте мне тикер акции, чтобы получить прогноз её цены.")


if __name__ == "__main__":

    application = Application.builder().token(TELEGRAM_TOKEN).build()


    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_ticker))


    print("Бот запущен. Ожидаю сообщения...")
    asyncio.get_event_loop().run_until_complete(application.run_polling())

Бот запущен. Ожидаю сообщения...
